In [1]:
import MySQLdb

conn = MySQLdb.connect(host='localhost', user='root', passwd='buaascse', db='future')
cursor = conn.cursor()

cursor.execute('select sum(done_price*qty) from chengjiao where date_format(tradedate,"%Y")="2014" group by capital_account_new')
turnover2014 = cursor.fetchall()
turnover2014 = [x[0]/10**5 for x in turnover2014]

cursor.execute('select sum(done_price*qty) from chengjiao where date_format(tradedate,"%Y")="2015" group by capital_account_new')
turnover2015 = cursor.fetchall()
turnover2015 = [x[0]/10**5 for x in turnover2015]

cursor.execute('select sum(done_price*qty) from chengjiao where date_format(tradedate,"%Y")="2016" group by capital_account_new')
turnover2016 = cursor.fetchall()
turnover2016 = [x[0]/10**5 for x in turnover2016]

cursor.close()
conn.close()

In [2]:
width = 50
bins2014 = int(max(turnover2014) / width) + 1
bins2015 = int(max(turnover2015) / width) + 1
bins2016 = int(max(turnover2016) / width) + 1
bins = max(bins2014, bins2015, bins2016)
data2014 = [0] * bins
data2015 = [0] * bins
data2016 = [0] * bins
for x in turnover2014:
    data2014[int(x/width)] += 1
for x in turnover2015:
    data2015[int(x/width)] += 1
for x in turnover2016:
    data2016[int(x/width)] += 1

In [4]:
from pyecharts import Bar
bar = Bar('用户交易额(十万)')
bar._width = 1000
attr = ['%d~%d'%(width*i,width*(i+1)) for i in range(bins)]
bar.add('2014年', attr[:50], data2014[:50], is_datazoom_show=True)
bar.add('2015年', attr[:50], data2015[:50], is_datazoom_show=True)
bar.add('2016年', attr[:50], data2016[:50], is_datazoom_show=True)
bar

In [5]:
import math

alpha2014 = 0
for x in turnover2014:
    alpha2014 += math.log(x)
alpha2014 = 1 + len(turnover2014) / alpha2014

alpha2015 = 0
for x in turnover2015:
    alpha2015 += math.log(x)
alpha2015 = 1 + len(turnover2015) / alpha2015

alpha2016 = 0
for x in turnover2016:
    alpha2016 += math.log(x)
alpha2016 = 1 + len(turnover2016) / alpha2016

print alpha2014, alpha2015, alpha2016

1.18512613023 1.18588470885 1.18913900782


In [7]:
pro2014 = []
pro2015 = []
pro2016 = []
for i in range(bins):
    pro2014.append((i*width+1)**(1-alpha2014) - (i*width+width+1)**(1-alpha2014))
    pro2015.append((i*width+1)**(1-alpha2015) - (i*width+width+1)**(1-alpha2015))
    pro2016.append((i*width+1)**(1-alpha2016) - (i*width+width+1)**(1-alpha2016))

In [8]:
from scipy.stats import chisquare
print chisquare(data2014, [x*len(turnover2014) for x in pro2014])
print chisquare(data2015, [x*len(turnover2015) for x in pro2015])
print chisquare(data2016, [x*len(turnover2016) for x in pro2016])

Power_divergenceResult(statistic=8473.391807052174, pvalue=1.0)
Power_divergenceResult(statistic=8597.2672048830864, pvalue=1.0)
Power_divergenceResult(statistic=8307.0809609519456, pvalue=1.0)
